In [1]:
#You need the Data set from kaggle to run this
import pandas as pd
import numpy as np
# importing plot libraries
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# Training models import statements
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# Importing tensorflow requirements
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
weather_data = './WeatherEvents_Jan2016-Dec2021.csv'
data = pd.read_csv(weather_data)
df = pd.DataFrame(data)
newdf = df.drop('AirportCode',axis='columns')
tempdf = newdf
print('Setup Complete')

Setup Complete


In [2]:
print('\nNumber of rows and columns in the data set: ', newdf.shape)


Number of rows and columns in the data set:  (7479165, 13)


In [3]:
newdf.rename(columns={'EventId':'Year'},inplace=True)

In [4]:
datetimeFormat = '%Y-%m-%d %H:%M:%S'
newdf['StartTime(UTC)'] = pd.to_datetime(newdf['StartTime(UTC)'], format=datetimeFormat)
newdf['EndTime(UTC)'] = pd.to_datetime(newdf['EndTime(UTC)'], format=datetimeFormat)
newdf['Year'] = newdf['StartTime(UTC)'].dt.year
newdf['Duration'] = newdf['EndTime(UTC)'] - newdf['StartTime(UTC)']
#Formatting the Duration Data
newdf['Duration'] = newdf['Duration'].dt.total_seconds()
newdf['Duration'] = newdf['Duration']/(60*60)
newdf = newdf[(newdf['Duration']< 30*24) & (newdf['Duration'] != 0)]

In [5]:
newdf.columns

Index(['Year', 'Type', 'Severity', 'StartTime(UTC)', 'EndTime(UTC)',
       'Precipitation(in)', 'TimeZone', 'LocationLat', 'LocationLng', 'City',
       'County', 'State', 'ZipCode', 'Duration'],
      dtype='object')

In [6]:
newdf = newdf.dropna(axis=0)

In [7]:
newdf.isna().sum()

Year                 0
Type                 0
Severity             0
StartTime(UTC)       0
EndTime(UTC)         0
Precipitation(in)    0
TimeZone             0
LocationLat          0
LocationLng          0
City                 0
County               0
State                0
ZipCode              0
Duration             0
dtype: int64

In [8]:
newdf.dtypes

Year                          int64
Type                         object
Severity                     object
StartTime(UTC)       datetime64[ns]
EndTime(UTC)         datetime64[ns]
Precipitation(in)           float64
TimeZone                     object
LocationLat                 float64
LocationLng                 float64
City                         object
County                       object
State                        object
ZipCode                     float64
Duration                    float64
dtype: object

In [9]:
newdf.groupby(['Year', 'Type', 'Severity', 'StartTime(UTC)', 'EndTime(UTC)',
       'Precipitation(in)', 'TimeZone', 'LocationLat', 'LocationLng', 'City',
       'County', 'State', 'ZipCode'],)


In [10]:
newdf.head()

,Year,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,LocationLat,LocationLng,City,County,State,ZipCode,Duration
0,2016,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,1.333333
1,2016,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,0.666667
2,2016,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,9.666667
3,2016,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,0.333333
4,2016,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0,2.000000


In [11]:
newdf.tail()

,Year,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,LocationLat,LocationLng,City,County,State,ZipCode,Duration
7479160,2021,Storm,Severe,2021-12-26 18:19:00,2021-12-26 18:29:00,0.00,US/Mountain,42.5833,-108.2833,Lander,Fremont,WY,82520.0,0.166667
7479161,2021,Snow,Light,2021-12-26 18:29:00,2021-12-26 18:38:00,0.02,US/Mountain,42.5833,-108.2833,Lander,Fremont,WY,82520.0,0.150000
7479162,2021,Snow,Light,2021-12-28 00:53:00,2021-12-28 02:27:00,0.08,US/Mountain,42.5833,-108.2833,Lander,Fremont,WY,82520.0,1.566667
7479163,2021,Snow,Moderate,2021-12-28 02:27:00,2021-12-28 03:09:00,0.09,US/Mountain,42.5833,-108.2833,Lander,Fremont,WY,82520.0,0.700000
7479164,2021,Snow,Light,2021-12-28 03:09:00,2021-12-28 03:19:00,0.01,US/Mountain,42.5833,-108.2833,Lander,Fremont,WY,82520.0,0.166667


In [12]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7419580 entries, 0 to 7479164
Data columns (total 14 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Year               int64         
 1   Type               object        
 2   Severity           object        
 3   StartTime(UTC)     datetime64[ns]
 4   EndTime(UTC)       datetime64[ns]
 5   Precipitation(in)  float64       
 6   TimeZone           object        
 7   LocationLat        float64       
 8   LocationLng        float64       
 9   City               object        
 10  County             object        
 11  State              object        
 12  ZipCode            float64       
 13  Duration           float64       
dtypes: datetime64[ns](2), float64(5), int64(1), object(6)
memory usage: 849.1+ MB


In [13]:
# Code cell for occurence of prediction type on specific city & county:
def getOccur(type,city,county,state):
    totalOcur = []
    i = 2016
    while i <= 2021:
        group1 = newdf.groupby(['Year','Type','City','County','State'])
        group2 = group1.get_group((i,type,city,county,state))
        total = len(group2)
        totalOcur.append(total)
        i+=1
    return totalOcur
trainList1 = getOccur('Rain','Lander','Fremont','WY')
print(trainList1)
traindf = pd.DataFrame(trainList1, columns=['Year 2016-21'])
x = traindf.values
y = [123,456,231,333,111,267]
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.2, random_state=42)

[398, 308, 328, 306, 98, 262]


In [14]:
testList1 = [76]
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(xtrain, ytrain)
features = np.array(testList1)
prediction1 = model.predict(features.reshape(-1,1))
print(int(prediction1))

151


In [15]:
# Code cell for Country Weather type occurence prediction:
def couOcur(type):
    i = 2016
    totalOcu = []
    while i <= 2021:
        group0 = newdf.groupby(['Year','Type'])
        group1 = group0.get_group((i,type))
        total = len(group1)
        totalOcu.append(total)
        i+=1
    return totalOcu
trainList2 = couOcur('Rain')
print(trainList2)
trainList2df = pd.DataFrame(trainList2, columns=['Year 2016-21'])
x1 = trainList2df.values
y1 = [100000,200045,567789,123456,546678,321123]
xtrain1, xtest1, ytrain1, ytest1 = train_test_split(x1,y1, test_size=0.2, random_state=42)

[726106, 770223, 830943, 725458, 681195, 640953]


In [16]:
# RandomForestRegressor model
testList2 = [76672]
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(xtrain1, ytrain1)
features = np.array(testList2)
prediction1 = model.predict(features.reshape(-1,1))
print(int(prediction1))

367349


In [22]:
#Experimenting with cross reference form input
group0 = newdf.groupby(['Type','City','County','State']).groups.keys()
finalKeys = set(group0)
print(finalKeys)

{('Cold', 'Harvey', 'Wells', 'ND'), ('Hail', 'Williams', 'Coconino', 'AZ'), ('Rain', 'Barnwell', 'Barnwell', 'SC'), ('Fog', 'Saint George', 'Washington', 'UT'), ('Cold', 'La Verne', 'Los Angeles', 'CA'), ('Precipitation', 'Eastsound', 'San Juan', 'WA'), ('Snow', 'Florien', 'Sabine', 'LA'), ('Fog', 'Athens', 'Clarke', 'GA'), ('Fog', 'Poughkeepsie', 'Dutchess', 'NY'), ('Snow', 'Mayfield', 'Graves', 'KY'), ('Storm', 'Tucson', 'Pima', 'AZ'), ('Fog', 'Wolf Creek', 'Josephine', 'OR'), ('Precipitation', 'Graham', 'Young', 'TX'), ('Fog', 'Sulphur Springs', 'Hopkins', 'TX'), ('Snow', 'Owosso', 'Shiawassee', 'MI'), ('Storm', 'Latrobe', 'Westmoreland', 'PA'), ('Precipitation', 'Aberdeen', 'Brown', 'SD'), ('Hail', 'Cookstown', 'Burlington', 'NJ'), ('Rain', 'Hillsville', 'Carroll', 'VA'), ('Fog', 'Evanston', 'Uinta', 'WY'), ('Storm', 'Green', 'Summit', 'OH'), ('Hail', 'Fort Huachuca', 'Cochise', 'AZ'), ('Cold', 'Greensboro', 'Guilford', 'NC'), ('Fog', 'Travis Afb', 'Solano', 'CA'), ('Precipitation'

In [18]:
#State keys:
group1 = newdf.groupby(['State']).groups.keys()
stateKeys = list(group1)


In [19]:
#Type keys:
group2 = newdf.groupby(['Type']).groups.keys()
typeKeys = list(group2)

In [20]:
#County Keys:
group3 = newdf.groupby(['County']).groups.keys()
countyKeys = list(group3)


In [21]:
group4 = newdf.groupby(['City']).groups.keys()
cityKeys = list(group4)